# Loop Workflow Example 3

* High level approach to making a 3D model from just a bounding box and source files as input. (In Australia only for now. Documentation to come)
* This notebook uses example data provided by the Geological Survey of Western Australia.
* To run with your own data you will need to run the Utility 1 notebook to create an hjson config file and supply your own dtm model (service or geotif file in local coordinate system)

## Map2Loop

In [ ]:
import os
from map2loop.project import Project
from map2loop.m2l_enums import VerboseLevel
from map2loop.m2l_enums import Datatype
from map2loop.sampler import SamplerSpacing, SamplerDecimator
from map2loop.sorter import SorterUseHint, SorterUseNetworkX, SorterAgeBased, SorterAlpha
import time

from datetime import datetime
nowtime=datetime.now().isoformat(timespec='minutes')   
model_name=nowtime.replace(":","-").replace("T","-")
loop_project_filename = os.path.join(model_name, "local_source.loop3d")

t0 = time.time()

bounding_box = {
    "minx": 520000,
    "miny": 7490000,
    "maxx": 550000,
    "maxy": 7510000,
    "base": -3200,
    "top": 1200,
}
proj = Project( 
    geology_filename = "./source_data/geol_clip.shp",
    fault_filename = "./source_data/faults_clip.shp",
    fold_filename = "./source_data/folds_clip.shp",
    structure_filename = "./source_data/structure_clip.shp",
    dtm_filename = './source_data/dtm_rp.tif',
    config_filename = './source_data/example.hjson',
    clut_filename = './source_data/500kibg_colours.csv',
    clut_file_legacy = True,
    verbose_level = VerboseLevel.NONE,
    tmp_path = model_name,
    working_projection = "EPSG:28350",
    bounding_box = bounding_box,
    loop_project_filename = loop_project_filename
)
proj.set_minimum_fault_length(5000.0)
proj.set_sampler(Datatype.GEOLOGY, SamplerSpacing(50.0))
proj.set_sampler(Datatype.STRUCTURE, SamplerDecimator(2))
proj.map_data.config.geology_config["intrusive_text"] = "mafic intrusive"
proj.set_ignore_codes(["cover", "Fortescue_Group", "A_FO_od"])

column = [
    # youngest
    'Turee_Creek_Group',
    'Boolgeeda_Iron_Formation',
    'Woongarra_Rhyolite',
    'Weeli_Wolli_Formation',
    'Brockman_Iron_Formation',
    'Mount_McRae_Shale_and_Mount_Sylvia_Formation',
    'Wittenoom_Formation',
    'Marra_Mamba_Iron_Formation',
    'Jeerinah_Formation',
    'Bunjinah_Formation',
    'Pyradie_Formation',
    'Fortescue_Group',
    # oldest
]
# proj.set_sorter(SorterUseNetworkX())
proj.set_sorter(SorterAlpha())
proj.run_all(take_best=True)
# proj.run_all(user_defined_stratigraphic_column=column)

t1 = time.time()

## Loop Structural

In [ ]:
import LoopProjectFile as LPF
from LoopStructural.visualisation import LavaVuModelViewer
from LoopStructural.modelling.input.project_file import LoopProjectfileProcessor as LPFProcessor
import LoopStructural
import numpy as np
from scipy.interpolate import RegularGridInterpolator
from osgeo import gdal
import pandas as pd

t2 = time.time()

LPFilename = loop_project_filename
fault_params = {
    'interpolatortype':'FDI',
    'nelements':1e4,
}
foliation_params = {
    'interpolatortype':'FDI' , # 'interpolatortype':'PLI',
    'nelements':1e5,  # how many tetras/voxels
    'regularisation':5,
}

projFile = LPF.ProjectFile(LPFilename)
processedData = LPFProcessor(projFile)
processedData.foliation_properties['sg'] = foliation_params
processedData.fault_properties['interpolatortype'] = fault_params['interpolatortype']
processedData.fault_properties['nelements'] = fault_params['nelements']

model = LoopStructural.GeologicalModel.from_processor(processedData)
model.update()

clip_on_dtm=True
if(clip_on_dtm):
    bounding_box = proj.map_data.get_bounding_box()
    minx = bounding_box['minx']
    maxx = bounding_box['maxx']
    miny = bounding_box['miny']
    maxy = bounding_box['maxy']
    model_base = bounding_box['base']
    model_top = bounding_box['top']
    dtm = gdal.Open(os.path.join("m2l_data_tmp", "StupidGDALLocalFile.tif"))
    dtm_val = dtm.GetRasterBand(1).ReadAsArray().T
    # Convert bounds to gdal raster bounds
    x=np.linspace(minx,maxx,dtm_val.shape[0])
    y=np.linspace(miny,maxy,dtm_val.shape[0])
    dtm_interpolator = RegularGridInterpolator((x,y),dtm_val)
    model.dtm = lambda xyz : dtm_interpolator(xyz[:,:2])

vtk_path = model_name+'/vtk/'
filename = os.path.join(model_name,'vtk','surface_name_{}.vtk')
view = LavaVuModelViewer(model)
view.nsteps=np.array([50,50,50])
for sg in model.feature_name_index:
    if( 'super' in sg):
        view.add_data(model.features[model.feature_name_index[sg]])
view.nelements = 1e5
view.add_model_surfaces(filename=filename,faults=False)
view.nelements=1e6
view.add_model_surfaces(filename=filename,strati=False,displacement_cmap = 'rainbow')
view.lv.webgl(vtk_path+model_name)
view.nsteps = np.array([200,200,200])
if(clip_on_dtm):
    colours = list(pd.DataFrame(data=proj.stratigraphic_column.column,columns=["name"]).merge(proj.stratigraphic_column.stratigraphicUnits[["name","colour"]], on="name")["colour"])
    colours.reverse()
    view.add_dtm(paint_with=lambda xyz: model.evaluate_model(xyz,scale=False), cmap=colours)
view.add_model()

view.lv.control.Range('alpha', label="Global Opacity")
view.lv.control.DualRange(['xmin', 'xmax'], label="x clip", step=0.01, values=[0.0,1.0])
view.lv.control.DualRange(['ymin', 'ymax'], label="y clip", step=0.01, values=[0.0,1.0])
view.lv.control.DualRange(['zmin', 'zmax'], label="z clip", step=0.01, values=[0.0,1.0])
view.lv.control.Range(command='background', range=(0,1), step=0.1, value=0.8)
view.lv.control.show() #Show the control panel, including the viewer window
view.interactive()

t3 = time.time()

## Elapsed Time

In [ ]:
# Print element and total processing time
m2l_time = t1-t0
ls_time = t3-t2
total = t3-t0

m2l_string = f"{m2l_time} sec" if m2l_time < 60 else f"{m2l_time/60.0} min"
ls_string = f"{ls_time} sec" if ls_time < 60 else f"{ls_time/60.0} min"
total_string = f"{total} sec" if total < 60 else f"{total/60.0} min"
print(f"map2loop {m2l_string}, LoopStructural {ls_string}, Total {total_string}")

In [ ]:
# Draw overlay of point data on geology map
# (options are 'basal_contacts', contacts','orientations','faults')
proj.draw_geology_map(overlay="basal_contacts")

In [ ]:
# Extract estimate of the stratigraphic column
print(proj.stratigraphic_column.column)